In [0]:
from databricks.vector_search.client import VectorSearchClient

# Initialize client
vsc = VectorSearchClient()

# Get index
index = vsc.get_index(index_name="docai-dbx.gold.doc_embeddings_index")

# Use the built-in embedding function for querying (same one used for index creation)
results = index.similarity_search(
    query_text="What is the refund policy?",
    columns=["chunk_id", "file_name", "text_chunk"],
    num_results=5
)
display(results)
# Show results
# for r in results['result']:
#     print(f"📄 File: {r['file_name']}")
#     print(f"📝 Chunk: {r['text_chunk'][:300]}...\n---\n")


In [0]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_id = "microsoft/Phi-3-mini-128k-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")

qa_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Prepare prompt from retrieved chunks
context = "\n".join([r["text_chunk"] for r in results["result"]])

prompt = f"""Use the following document excerpts to answer the question.

Document:
{context}

Question: What is the refund policy?
Answer:"""

response = qa_pipeline(prompt, max_new_tokens=200, do_sample=True)[0]["generated_text"]
answer = response.split("Answer:")[-1].strip()

print("\n🤖 Final Answer:\n")
print(answer)
